<a href="https://colab.research.google.com/github/cokoroafor11/Playlist-Recommender-App/blob/main/recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building a Spotify Recommendation System

## Imports and Credentials

In [1]:
!pip install spotipy --upgrade
!pip install gdown
!pip install vaderSentiment
!pip install swifter

import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from ast import literal_eval
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from google.colab import files
import swifter
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 20.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.6/490.6 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.3.5-py3-none-any.whl size=16518 sha256=551318a579a1a1a6edf5fdb283a6317a8ad9eeb020b957d0dab644eebb290627
  Stored in directory: /root/.cache/pip/wheels/00/7e/8c/438ea309a16ef1a9130849060f962e0d2

In [2]:
#Save client credentials
client_id = "c1f74565be774e65aa211462aaf5fed8"
client_secret = "2edce4052f8f46639c0e112658572d66"
#Create object
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,client_secret=client_secret),requests_timeout=100,retries=3)

## Feature Database Import

In [3]:
def excel_list_to_df(file):
    '''Create the dataframe from the excel sheet of playlists'''

    df = pd.read_excel(file)
    #playlists = df['Link']
    return df

In [4]:
!gdown 'https://docs.google.com/uc?id=1xq_oFsO6s1OgiRYFzyTv2lXy0pghUJ0Y&export=download'
features_df = excel_list_to_df('/content/spotify_features.xlsx')

Downloading...
From: https://docs.google.com/uc?id=1xq_oFsO6s1OgiRYFzyTv2lXy0pghUJ0Y&export=download
To: /content/spotify_features.xlsx
100% 132k/132k [00:00<00:00, 106MB/s]


## Functions for Creating Feature Set

In [5]:
def extract_songs(playlist):
    #Make sure playlist input is string
    if type(playlist) != str:
        playlist = str(playlist)

    results = spotify.playlist_items(playlist)
    tracks = results['items']
    uris = []
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])

    #Append tracks to list if no type error
    for elem in tracks:
        try:
            uris.append(elem['track']['uri'])
        except TypeError:
            continue
    return uris

def get_artist_info(track):
    '''Helper function for get_track_info function'''
    artist_info = {}
    artist_id = track["artists"][0]["id"]
    artist = spotify.artist(artist_id)
    artist_pop = artist["popularity"]
    artist_genres = artist["genres"]

    artist_info['artist_genres'] = artist_genres
    artist_info['artist_popularity'] = artist_pop
    return artist_info

def get_track_info(track):
    track = spotify.track(track)
    track_info = {}
    artists= []
    artist_info = get_artist_info(track)
    track_info['name'] = track['name']
    for artist in track['artists']:
        artists.append(artist['name'])
    track_info['artists'] = artists
    track_info['popularity'] = track['popularity']
    return track_info, artist_info

def populate_song_info(playlist):
    '''
    Function that populates an dictionary where keys are the audio features and values are an array of scores for each song
    This function performs the lionshare of data setup
    '''
    songlist = extract_songs(playlist)

    #Storage for various types of data
    song_data = {}
    name_array = []
    artist_array = []
    song_pop_array = []
    artist_pop_array = []
    genres_array = []
    dance_array = []
    energy_array = []
    key_array = []
    loudness_array = []
    mode_array = []
    speech_array = []
    acoustic_array = []
    instrument_array = []
    live_array = []
    valence_array = []
    tempo_array = []
    link_array = []

    #Loop to append song info to appropriate array
    for song in songlist:

        features = spotify.audio_features(song)[0]
        #Check if features don't exist, skip the song entry
        if features == None:
            continue

        #Get song names, artists, popularity, and genres
        track, artist = get_track_info(song)
        name_array.append(track['name'])
        artist_array.append(track['artists'])
        song_pop_array.append(track['popularity'])
        artist_pop_array.append(artist['artist_popularity'])
        genres_array.append(artist['artist_genres'])

        #Append feature to each corresponding array
        dance_array.append(features['danceability'])
        energy_array.append(features['energy'])
        key_array.append(features['key'])
        loudness_array.append(features['loudness'])
        mode_array.append(features['mode'])
        speech_array.append(features['speechiness'])
        acoustic_array.append(features['acousticness'])
        instrument_array.append(features['instrumentalness'])
        live_array.append(features['liveness'])
        valence_array.append(features['valence'])
        tempo_array.append(features['tempo'])
        link_array.append(song)

    #Put all song data in a library with proper labels
    song_data['track_name'] = name_array
    song_data['artists'] = artist_array
    song_data['song_popularity'] = song_pop_array
    song_data['artist_popularity'] = artist_pop_array
    song_data['genres'] = genres_array
    song_data['danceability'] = dance_array
    song_data['energy'] = energy_array
    song_data['key'] = key_array
    song_data['loudness'] = loudness_array
    song_data['mode'] = mode_array
    song_data['speechiness'] = speech_array
    song_data['acousticness'] = acoustic_array
    song_data['instrumentalness'] = instrument_array
    song_data['liveness'] = live_array
    song_data['valence'] = valence_array
    song_data['tempo'] = tempo_array
    song_data['link'] = link_array

    song_db = pd.DataFrame(song_data)
    return song_db

## Data Pre-Processing

In [6]:
features_df.dropna(inplace=True)
features_df

,Unnamed: 0,track_name,artists,song_popularity,artist_popularity,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,link
0,0,Attention,['Doja Cat'],74,85,"['dance pop', 'pop']",0.759,0.567,2,-7.911,1,0.2480,0.1180,0.027800,0.1050,0.421,87.981,spotify:track:11xC6P3iKYpFThT6Ce1KdG
1,1,Popular (with Playboi Carti & Madonna) - The I...,"['The Weeknd', 'Playboi Carti', 'Madonna']",89,95,"['canadian contemporary r&b', 'canadian pop', ...",0.852,0.675,1,-6.271,1,0.1970,0.0652,0.000037,0.3600,0.830,99.012,spotify:track:5xP9lQYA8YQmQh6BOxcAnR
2,2,Calm Down (with Selena Gomez),"['Rema', 'Selena Gomez']",95,81,"['afrobeats', 'nigerian pop']",0.801,0.806,11,-5.206,1,0.0381,0.3820,0.000669,0.1140,0.802,106.999,spotify:track:0WtM2NBVQNNJLh6scP13H8
3,3,Flowers,['Miley Cyrus'],97,87,['pop'],0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,spotify:track:0yLdNVWF3Srea0uzk55zFn
4,4,Calling (Spider-Man: Across the Spider-Verse) ...,"['Metro Boomin', 'Swae Lee', 'NAV', 'A Boogie ...",91,91,['rap'],0.631,0.535,0,-7.836,1,0.0842,0.4640,0.000000,0.1150,0.220,140.127,spotify:track:5rurggqwwudn9clMdcchxT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,903,Something's Gonna Kill Me,['Corey Kent'],73,64,"['modern country pop', 'texas country']",0.659,0.823,4,-5.770,0,0.0415,0.0174,0.000002,0.1100,0.405,112.931,spotify:track:41cv1emXVm5Su4DWvltXa6
838,904,IF YOU GO DOWN (I'M GOIN' DOWN TOO),['Kelsea Ballerini'],80,72,"['contemporary country', 'country']",0.676,0.856,11,-4.350,1,0.0393,0.4540,0.000000,0.1410,0.965,144.092,spotify:track:1jX9wlAgVkAl0fCZqkCI73
839,905,Bury Me in Georgia - Single Edit,['Kane Brown'],75,75,"['black americana', 'contemporary country', 'c...",0.391,0.816,2,-5.746,0,0.0660,0.0169,0.000000,0.0893,0.457,160.001,spotify:track:2mltIbKkhQqyh8gCzaqxqA
840,906,Memory Lane,['Old Dominion'],81,71,"['contemporary country', 'country', 'country r...",0.645,0.800,6,-4.272,1,0.0363,0.0189,0.000011,0.0858,0.375,118.977,spotify:track:5kB6WithhB8Iv08EMNmAbt


In [7]:
features_df.dtypes

Unnamed: 0             int64
track_name            object
artists               object
song_popularity        int64
artist_popularity      int64
genres                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
link                  object
dtype: object

In [8]:
features_df.drop(columns = ['Unnamed: 0'],inplace=True)

In [9]:
features_df.iloc[0]['genres'][0]

'['

In [10]:
features_df['genres'] = features_df['genres'].apply(lambda row: literal_eval(row))

In [11]:
features_df.iloc[0]['genres'][0]

'dance pop'

In [12]:
features_df.iloc[0]['artists'][0]

'['

In [13]:
features_df['artists'] = features_df['artists'].apply(lambda row: literal_eval(row))

In [14]:
features_df.iloc[0]['artists'][0]

'Doja Cat'

## Feature Engineering

### Sentiment Analysis

In [15]:
def sent_analysis(song_title):
  '''
  The sentiment analysis outputs positive, negative, neutral scores
  Combines into compound score that is a combo of all the above
  We will use the compound score, then put it into one of three buckets
  '''
  analyzer = SentimentIntensityAnalyzer()
  analysis = analyzer.polarity_scores(song_title)['compound']
  if analysis < -0.5:
    return 'negative'
  elif analysis > 0.5:
    return 'positive'
  else:
    return 'neutral'

In [16]:
sent_analysis(features_df['track_name'][0])

'neutral'

In [17]:
'''
start = time.time()
features_df['track_name'].swifter.apply(lambda row: sent_analysis(row)['compound'])
end = time.time()
print(end-start)
'''

"\nstart = time.time()\nfeatures_df['track_name'].swifter.apply(lambda row: sent_analysis(row)['compound'])\nend = time.time()\nprint(end-start)\n"

In [18]:
features_df['sentiment'] = features_df['track_name'].apply(lambda row: sent_analysis(row))

In [19]:
features_df['sentiment']

0       neutral
1      positive
2       neutral
3       neutral
4       neutral
         ...   
837    negative
838     neutral
839     neutral
840     neutral
841     neutral
Name: sentiment, Length: 842, dtype: object

In [20]:
features_df['sentiment'].describe()

count         842
unique          3
top       neutral
freq          750
Name: sentiment, dtype: object

### Normalization

#### Float Columns

In [21]:
float_cols = features_df.select_dtypes(include=['float64'])
scaler = MinMaxScaler()
float_df = pd.DataFrame(scaler.fit_transform(float_cols), columns = float_cols.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(float_df.iteritems()))

#### Popularity Columns

In [22]:
pop_cols = features_df[['song_popularity','artist_popularity']]
scaler = MinMaxScaler()
pop_df = pd.DataFrame(scaler.fit_transform(pop_cols), columns = pop_cols.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(pop_df.iteritems()))

### Key Column

In [23]:
key = features_df[['key']]
scaler = MinMaxScaler()
key_df = pd.DataFrame(scaler.fit_transform(key), columns = key.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(key_df.iteritems()))

### One Hot Encoding

In [24]:
def one_hot(df, column):
  if type(column) != str: column = str(column)
  df = pd.get_dummies(df, columns=[column])
  return df

features_df = one_hot(features_df, 'sentiment')

### TF-IDF for Genres

In [25]:
#tfidf genre lists
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(features_df['genres'].apply(lambda x: " ".join(x)))
genres_df = pd.DataFrame(tfidf_matrix.toarray())
genres_df.columns = ['genre' + "_" + i for i in tfidf.get_feature_names_out()]
genres_df.reset_index(drop = True, inplace=True)

In [26]:
genres_df

,genre_150,genre_acid,genre_acoustic,genre_afro,genre_afrobeats,genre_afrofuturism,genre_age,genre_agronejo,genre_album,genre_alt,...,genre_us,genre_vapor,genre_venezolano,genre_viral,genre_wave,genre_west,genre_wyoming,genre_yacht,genre_york,genre_zolo
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.662669,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
839,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
features_df = pd.concat([features_df,genres_df],axis = 1)
features_df.drop(columns = ['genres'], inplace = True)

In [28]:
features_df

,track_name,artists,song_popularity,artist_popularity,danceability,energy,key,loudness,mode,speechiness,...,genre_us,genre_vapor,genre_venezolano,genre_viral,genre_wave,genre_west,genre_wyoming,genre_yacht,genre_york,genre_zolo
0,Attention,[Doja Cat],0.48,0.765625,0.719577,0.551255,0.181818,0.593736,1,0.364483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Popular (with Playboi Carti & Madonna) - The I...,"[The Weeknd, Playboi Carti, Madonna]",0.78,0.921875,0.842593,0.664226,0.090909,0.683846,1,0.281163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Calm Down (with Selena Gomez),"[Rema, Selena Gomez]",0.90,0.703125,0.775132,0.801255,1.000000,0.742363,1,0.021565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Flowers,[Miley Cyrus],0.94,0.796875,0.650794,0.670502,0.000000,0.790769,1,0.068453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Calling (Spider-Man: Across the Spider-Verse) ...,"[Metro Boomin, Swae Lee, NAV, A Boogie Wit da ...",0.82,0.859375,0.550265,0.517782,0.000000,0.597857,1,0.096880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,Something's Gonna Kill Me,[Corey Kent],0.46,0.437500,0.587302,0.819038,0.363636,0.711374,0,0.027120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
838,IF YOU GO DOWN (I'M GOIN' DOWN TOO),[Kelsea Ballerini],0.60,0.562500,0.609788,0.853556,1.000000,0.789396,1,0.023526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
839,Bury Me in Georgia - Single Edit,[Kane Brown],0.50,0.609375,0.232804,0.811715,0.181818,0.712692,0,0.067146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
840,Memory Lane,[Old Dominion],0.62,0.546875,0.568783,0.794979,0.545455,0.793681,1,0.018624,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
features_df['link']

0      spotify:track:11xC6P3iKYpFThT6Ce1KdG
1      spotify:track:5xP9lQYA8YQmQh6BOxcAnR
2      spotify:track:0WtM2NBVQNNJLh6scP13H8
3      spotify:track:0yLdNVWF3Srea0uzk55zFn
4      spotify:track:5rurggqwwudn9clMdcchxT
                       ...                 
837    spotify:track:41cv1emXVm5Su4DWvltXa6
838    spotify:track:1jX9wlAgVkAl0fCZqkCI73
839    spotify:track:2mltIbKkhQqyh8gCzaqxqA
840    spotify:track:5kB6WithhB8Iv08EMNmAbt
841    spotify:track:6BOP0cv1eeXcvi1oE8bDVZ
Name: link, Length: 842, dtype: object

## User Playlist Feature Generation

In [30]:
#process to go from playlist to feature vector
#populate_song_info()
#sent_analysis
#one hot encode sent analysis

In [31]:
def gen_user_playlist_features(playlist):
  #Get basic feature df for users
  user_df = populate_song_info(playlist)

  #Sentiment Analysis
  user_df['sentiment'] = user_df['track_name'].apply(lambda row: sent_analysis(row))

  #Normalization
  ##Float columns
  #Get columns to scale
  float_cols = user_df.select_dtypes(include=['float64'])
  scaler = MinMaxScaler()
  float_df = pd.DataFrame(scaler.fit_transform(float_cols), columns = float_cols.columns)
  #Insert scaled columns into df
  user_df= user_df.assign(**dict(float_df.iteritems()))
  ##Key and Popularity Columns
  #Get columns to scale
  pop_key_cols = user_df[['song_popularity','artist_popularity','key']]
  scaler = MinMaxScaler()
  pop_key_df = pd.DataFrame(scaler.fit_transform(pop_key_cols), columns = pop_key_cols.columns)
  #Insert scaled columns into df
  user_df= user_df.assign(**dict(pop_df.iteritems()))

  #One hot encoding
  user_df = one_hot(user_df, 'sentiment')

  #Tfidf genre lists
  tfidf = TfidfVectorizer()
  tfidf_matrix =  tfidf.fit_transform(user_df['genres'].apply(lambda x: " ".join(x)))
  genres_df = pd.DataFrame(tfidf_matrix.toarray())
  genres_df.columns = ['genre' + "_" + i for i in tfidf.get_feature_names_out()]
  genres_df.reset_index(drop = True, inplace=True)
  user_df.drop(columns = ['genres'], inplace = True)
  user_df = pd.concat([user_df,genres_df],axis = 1)
  user_df.dropna(inplace=True)
  return user_df


In [32]:
user_playlist = gen_user_playlist_features('https://open.spotify.com/playlist/5PJUOcZUy72vVk3OW54nX8?si=162672425675420f')

In [33]:
#Get features of songs that aren't in this current playlist
def get_non_playlist_features(features_df, user_playlist_df):
  df_all = features_df.merge(user_playlist_df['link'], on=['link'],suffixes=('', '_x'), how='left', indicator=True)
  no_playlist_df = df_all[df_all['_merge'] == 'left_only']
  no_playlist_df = no_playlist_df.drop('_merge', axis=1)
  no_playlist_df.drop(list(no_playlist_df.filter(regex = '_x')), axis = 1, inplace = True)
  return no_playlist_df


In [34]:
no_playlist_df = get_non_playlist_features(features_df,user_playlist)

In [70]:
def generate_recommendations(user_playlist, no_playlist_df,num_recommendations):
  cols = user_playlist.columns.union(no_playlist_df.columns)
  user_playlist = user_playlist.reindex(cols, axis=1, fill_value=0)
  no_playlist_df = no_playlist_df.reindex(cols,axis=1,fill_value=0)
  user_playlist_vector = user_playlist.drop(columns = ['track_name','artists','link']).sum()
  no_playlist_df['similarity'] = cosine_similarity(no_playlist_df.drop(columns = ['track_name','artists','link']).values,user_playlist_vector.values.reshape(1, -1))[:,0]
  no_playlist_df.sort_values('similarity',ascending = False, inplace = True)
  #return no_playlist_df.drop(columns = ['track_name','artists','link']).values,user_playlist_vector
  return no_playlist_df.head(num_recommendations)[['track_name','artists','link','similarity']]

In [71]:
generate_recommendations(user_playlist,no_playlist_df,20)

,track_name,artists,link,similarity
481,Spinnin,"[Connor Price, Bens]",spotify:track:5OzgCc3gQsqchX95k1C1Qa,0.731568
562,In The Stars,[Benson Boone],spotify:track:1ei3hzQmrgealgRKFxIcWn,0.679329
118,8 AM,"[Nicki Nicole, Young Miko]",spotify:track:7CSmXJNeArnwDfUmtP4Gve,0.677090
176,Comfortably Numb,[Pink Floyd],spotify:track:5HNCy40Ni5BZJFw1TKzRsC,0.672642
56,Johnny Dang (with Paul Wall & DRODi),"[That Mexican OT, Paul Wall, DRODi]",spotify:track:1kwPxt35W7cG3rryxR48PO,0.667351
439,Kernkraft 400 (A Better Day),"[Topic, A7S]",spotify:track:3kcKlOkQQEPVwxwljbGJ5p,0.665811
671,For Tonight,[Giveon],spotify:track:61Emqg95O9zo1GNOcyxq4Y,0.665707
318,Riders on the Storm,[The Doors],spotify:track:14XWXWv5FoCbFzLksawpEe,0.665265
85,SHAKE SUMN,[DaBaby],spotify:track:3FhZPYvMGSjFc6boVYKlCv,0.662061
60,Annihilate (Spider-Man: Across the Spider-Vers...,"[Metro Boomin, Swae Lee, Lil Wayne, Offset]",spotify:track:39MK3d3fonIP8Mz9oHCTBB,0.661712
